# Valuing European-Style Swaptions Across Models

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.rates import *
from financepy.utils import *
from financepy.products.rates.ibor_single_curve import IborSingleCurve
from financepy.market.curves.interpolator import InterpTypes

####################################################################
#  FINANCEPY BETA Version 0.350 - This build: 30 Apr 2024 at 22:32 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Building a Libor Discount Curve

In [3]:
value_dt = Date(28, 2, 2014)
settle_dt = Date(4, 3, 2014)

We have a vector of dates and zero rates

In [4]:
depoDCCType = DayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = IborDeposit(settle_dt, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
swap_type = SwapTypes.PAY
fixed_dcc_type = DayCountTypes.ACT_365F
fixed_freq_type = FrequencyTypes.SEMI_ANNUAL

swap = IborSwap(settle_dt, "3Y", swap_type, 0.00790, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "4Y", swap_type, 0.01200, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "5Y", swap_type, 0.01570, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "6Y", swap_type, 0.01865, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "7Y", swap_type, 0.02160, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "8Y", swap_type, 0.02350, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "9Y", swap_type, 0.02540, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "10Y", swap_type, 0.0273, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "15Y", swap_type, 0.0297, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "20Y", swap_type, 0.0316, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "25Y", swap_type, 0.0335, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "30Y", swap_type, 0.0354, fixed_freq_type, fixed_dcc_type); swaps.append(swap)

libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

In [5]:
exercise_dt = settle_dt.add_tenor("5Y")
swap_maturity_dt = exercise_dt.add_tenor("5Y")
swap_fixed_cpn = 0.040852
swap_fixed_freq_type = FrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = DayCountTypes.THIRTY_E_360_ISDA
swapFloatFrequencyType = FrequencyTypes.QUARTERLY
swapFloatDayCountType = DayCountTypes.ACT_360
swapNotional = ONE_MILLION
swap_type = SwapTypes.PAY  
cal_type = CalendarTypes.WEEKEND
bd_type = BusDayAdjustTypes.NONE
dg_type = DateGenRuleTypes.BACKWARD

In [6]:
swaption = IborSwaption(settle_dt,
                            exercise_dt,
                            swap_maturity_dt,
                            swap_type,
                            swap_fixed_cpn,
                            swap_fixed_freq_type,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            cal_type, 
                            bd_type,
                            dg_type)

## Valuation using Black's Model

In [7]:
model = Black(0.1533)

In [8]:
swaption.value(settle_dt, libor_curve, model)

23177.882452968337

## Valuation using Shifted Black

In [9]:
model = BlackShifted(0.1533, 0.008)

In [10]:
swaption.value(settle_dt, libor_curve, model)

27708.379573470575

##  Valuation using SABR

In [11]:
model = SABR(0.132, 0.5, 0.5, 0.5)

In [12]:
swaption.value(settle_dt, libor_curve, model)

104569.44966320758

## Valuation using Shifted SABR

In [13]:
model = SABRShifted(0.1, 0.2, 0.1, 0.15, -0.005)

In [14]:
swaption.value(settle_dt, libor_curve, model)

163421.29261737553

## Valuation using Hull-White

In [15]:
model = HWTree(0.01, 0.02)

In [16]:
swaption.value(settle_dt, libor_curve, model)

34265.48386017303

## Valuation using Black-Karasinski

In [17]:
model = BKTree(0.2, 0.05)

In [18]:
swaption.value(settle_dt, libor_curve, model)

23574.204474888083

## Internals

In [19]:
print(swaption)

OBJECT TYPE: IborSwaption
SETTLEMENT DATE: 04-MAR-2014
EXERCISE DATE: 04-MAR-2019
SWAP FIXED LEG TYPE: SwapTypes.PAY
SWAP MATURITY DATE: 04-MAR-2024
SWAP NOTIONAL: 1000000
FIXED cpn: 4.0852
FIXED FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT DAY COUNT: DayCountTypes.ACT_360
PV01: 4.160180806485688
FWD SWAP RATE: 4.087013720473123
FWD DF TO EXPIRY: 0.923742186114948


We can see that the forward swap rate almost equals the fixed cpn. The underlying swap is close to being ATM forward.

In [20]:
swaption.print_swap_fixed_leg()

START DATE: 04-MAR-2019
MATURITY DATE: 04-MAR-2024
COUPON (%): 4.0852
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA

PAYMENTS VALUATION:
+---------+-------------+----------+--------+---------+--------+----------+-----------+
| PAY_NUM |    PAY_dt   | NOTIONAL |  RATE  |   PMNT  |   DF   |    PV    |   CUM_PV  |
+---------+-------------+----------+--------+---------+--------+----------+-----------+
|    1    | 04-SEP-2019 | 1000000  | 4.0852 | 20426.0 | 0.908  | 18546.05 |  18546.05 |
|    2    | 04-MAR-2020 | 1000000  | 4.0852 | 20426.0 | 0.8926 | 18232.34 |  36778.39 |
|    3    | 04-SEP-2020 | 1000000  | 4.0852 | 20426.0 | 0.8744 | 17861.51 |  54639.89 |
|    4    | 04-MAR-2021 | 1000000  | 4.0852 | 20426.0 | 0.8569 | 17503.38 |  72143.27 |
|    5    | 04-SEP-2021 | 1000000  | 4.0852 | 20426.0 | 0.8405 | 17169.02 |  89312.29 |
|    6    | 04-MAR-2022 | 1000000  | 4.0852 | 20426.0 | 0.8248 | 16846.58 | 106158.88 |
|    7    | 04-SEP-2022 | 1000000  |

Copyright (c) 2020 Dominic O'Kane